In [2]:
from tensorflow import keras
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, Input
from tensorflow.keras.models import Model, Sequential, save_model, load_model
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

from sklearn.model_selection import StratifiedKFold, KFold

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.inception_v3 import InceptionV3

from keras.utils import np_utils

import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import os
import glob

import scikitplot as skplt
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings('ignore')

import pickle

In [3]:
fh = open('/Users/home/Documents/Respiratory Illness AI/datasets/official_datasets/asthma_copd_covid_healthy_224_224_3', 'rb')
dataset = pickle.load(fh)

In [4]:
input_shape = (224, 224, 3)

n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True)
X, Y = zip(*dataset)

X_data, Y_data = np.array(X), np.array(Y)

In [8]:
def create_model(input_shape):
    baseModel = ResNet50(weights=None, include_top=False, input_shape=input_shape)

    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(2, 2))(headModel)
    headModel = Flatten(name="flatten")(headModel)

    headModel = Dense(512, activation='relu')(headModel)
    headModel = Dropout(0.5)(headModel)

    headModel = Dense(512, activation='relu')(headModel)
    headModel = Dropout(0.5)(headModel)

    headModel = Dense(512, activation='relu')(headModel)
    headModel = Dropout(0.5)(headModel)

    headModel = Dense(4, activation="softmax")(headModel)

    model = Model(inputs=baseModel.input, outputs=headModel)

    for layer in baseModel.layers:
        layer.trainable = False
        
    return model 

In [9]:
def k_fold_validation(x_data, y_data, kfold, test_num, input_shape):
    index = 0
    for train_i, test_i in kfold.split(x_data,y_data):
        index += 1 
        
        x_train, x_val = x_data[train_i], x_data[test_i]
        y_train, y_val = y_data[train_i], y_data[test_i]
        
        x_train = np.array([x.reshape(input_shape) for x in x_train])
        x_val = np.array([x.reshape(input_shape) for x in x_val])
        #remember to change number of classes 
        y_train = np.array(np_utils.to_categorical(y_train, 4))
        y_val = np.array(np_utils.to_categorical(y_val, 4))

        model = create_model(input_shape)
        
        model.compile(optimizer="Adam", 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
        
        model.fit(x=x_train,
                  y=y_train,
                  epochs=25,
                  batch_size=15,
                  validation_data=(x_val, y_val))
        
        save_model(model, 
           f'{test_num}rESNet{index}', 
           overwrite=True, 
           include_optimizer=True, 
           save_format=None, 
           signatures=None, options=None, save_traces=True)
        

In [10]:
k_fold_validation(X_data, Y_data, kfold, 13, input_shape)

2022-03-27 21:10:27.291989: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-27 21:10:29.316570: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
19/19 [==============================] - 31s 1s/step - loss: 12.2312 - accuracy: 0.2734 - val_loss: 2.3275 - val_accuracy: 0.2143
Epoch 2/25
19/19 [==============================] - 26s 1s/step - loss: 10.0399 - accuracy: 0.3165 - val_loss: 5.6264 - val_accuracy: 0.2143
Epoch 3/25
19/19 [==============================] - 28s 2s/step - loss: 8.3060 - accuracy: 0.2446 - val_loss: 2.0399 - val_accuracy: 0.3143
Epoch 4/25
19/19 [==============================] - 27s 1s/step - loss: 3.5399 - accuracy: 0.2770 - val_loss: 1.4107 - val_accuracy: 0.3000
Epoch 5/25
19/19 [==============================] - 26s 1s/step - loss: 1.6798 - accuracy: 0.2590 - val_loss: 1.3724 - val_accuracy: 0.3429
Epoch 6/25
19/19 [==============================] - 28s 1s/step - loss: 1.4547 - accuracy: 0.2986 - val_loss: 1.4173 - val_accuracy: 0.3286
Epoch 7/25
19/19 [==============================] - 25s 1s/step - loss: 1.4003 - accuracy: 0.2698 - val_loss: 1.3914 - val_accuracy: 0.3714
Epoch 8/25
19/19 [

2022-03-27 21:23:58.650819: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 13rESNet1/assets
Epoch 1/25
19/19 [==============================] - 31s 1s/step - loss: 12.2058 - accuracy: 0.1799 - val_loss: 6.8456 - val_accuracy: 0.2286
Epoch 2/25
19/19 [==============================] - 25s 1s/step - loss: 11.1152 - accuracy: 0.2230 - val_loss: 4.4797 - val_accuracy: 0.2286
Epoch 3/25
19/19 [==============================] - 26s 1s/step - loss: 6.3438 - accuracy: 0.2554 - val_loss: 2.0981 - val_accuracy: 0.2571
Epoch 4/25
19/19 [==============================] - 24s 1s/step - loss: 4.0296 - accuracy: 0.2770 - val_loss: 1.6500 - val_accuracy: 0.2286
Epoch 5/25
19/19 [==============================] - 25s 1s/step - loss: 2.4933 - accuracy: 0.2950 - val_loss: 1.5587 - val_accuracy: 0.2714
Epoch 6/25
19/19 [==============================] - 25s 1s/step - loss: 2.0022 - accuracy: 0.2986 - val_loss: 1.5105 - val_accuracy: 0.2714
Epoch 7/25
19/19 [==============================] - 26s 1s/step - loss: 1.7852 - accuracy: 0.3165 - val_lo

19/19 [==============================] - 25s 1s/step - loss: 1.3756 - accuracy: 0.2867 - val_loss: 1.3707 - val_accuracy: 0.3043
Epoch 9/25
19/19 [==============================] - 26s 1s/step - loss: 1.3840 - accuracy: 0.2760 - val_loss: 1.3626 - val_accuracy: 0.3043
Epoch 10/25
19/19 [==============================] - 26s 1s/step - loss: 1.4106 - accuracy: 0.2724 - val_loss: 1.3564 - val_accuracy: 0.3768
Epoch 11/25
19/19 [==============================] - 25s 1s/step - loss: 1.3957 - accuracy: 0.2760 - val_loss: 1.3770 - val_accuracy: 0.2754
Epoch 12/25
19/19 [==============================] - 25s 1s/step - loss: 1.3959 - accuracy: 0.2616 - val_loss: 1.3504 - val_accuracy: 0.3333
Epoch 13/25
19/19 [==============================] - 26s 1s/step - loss: 1.4123 - accuracy: 0.3011 - val_loss: 1.3770 - val_accuracy: 0.3043
Epoch 14/25
19/19 [==============================] - 26s 1s/step - loss: 1.3736 - accuracy: 0.2688 - val_loss: 1.3716 - val_accuracy: 0.2899
Epoch 15/25
19/19 [=======